In [8]:
import os
import numpy as np
import scipy.io.wavfile
import librosa
from skimage.transform import resize
import imageio

# Constants
kPRE_EMPHASIS_COEFF = 0.97
sample_rate = 48000
max_duration = 0.9999583333333333
max_samples = int(max_duration * sample_rate)

def pad_signal(signal, target_len):
    num_zeros_needed = target_len - len(signal)
    if num_zeros_needed > 0:
        num_zeros_front = np.random.randint(num_zeros_needed)
        num_zeros_back = num_zeros_needed - num_zeros_front
        return np.pad(signal, (num_zeros_front, num_zeros_back), mode='constant')
    else:
        return signal

def pre_emphasis(signal):
    first_amp = signal[0]
    all_amps_without_first = signal[1:]
    all_amps_without_last = signal[:-1]
    emphasized_signal = np.append(first_amp, all_amps_without_first - kPRE_EMPHASIS_COEFF * all_amps_without_last)
    return emphasized_signal

def pipeline(signal):
    emphasized_signal = pre_emphasis(signal)
    
    lifted_mfcc = librosa.feature.mfcc(
        y=emphasized_signal.astype(float), 
        sr=sample_rate, 
        n_mfcc=12, 
        dct_type=2, 
        norm='ortho', 
        lifter=22,
        n_fft=int(sample_rate * 0.025),
        hop_length=int(sample_rate * 0.01),
        power=2,
        center=False,
        window='hamming',
        n_mels=40
    )

    return lifted_mfcc

def preprocess_and_save_as_image(wav_file_path, save_directory):
    _, signal = scipy.io.wavfile.read(wav_file_path)
    signal = pad_signal(signal, target_len=max_samples)
    
    # Display the original signal
    # Remove the display code

    # Display the signal after pre-emphasis
    # Remove the display code

    # Display the MFCC after processing
    # Remove the display code

    # Resize for compatibility with a pre-trained ResNet-34
    mfc_3d = resize(np.rollaxis(np.array([pipeline(signal)] * 3), 0, 3), (224, 224, 3))
    
    # Save the final image using imageio
    save_filename = os.path.join(save_directory, os.path.splitext(os.path.basename(wav_file_path))[0] + '.jpg')
    imageio.imwrite(save_filename, mfc_3d.astype(np.uint8))

# Example usage:
input_directory = 'dataset'
output_directory = 'mfc_data'

for filename in os.listdir(input_directory):
    if filename.endswith('.wav'):
        wav_file_path = os.path.join(input_directory, filename)
        preprocess_and_save_as_image(wav_file_path, output_directory)
